In [19]:
import pandas as pd
from datetime import datetime

In [2]:
def std_convert(str):
    if len(str) == 1:
        return f"0{str}"
    else:
        return str

In [47]:
# https://www.data.go.kr/data/15049591/fileData.do <= 표준산업분류코드 데이터
STD_TOBZ_M_df = pd.read_csv("dataset/고용노동부_표준산업분류코드.csv")
STD_TOBZ_M_df['고용업종코드(중분류)'] = STD_TOBZ_M_df['고용업종코드(중분류)'].astype(int).astype(str)
STD_TOBZ_M_df['고용업종코드(중분류)'] = STD_TOBZ_M_df['고용업종코드(중분류)'].apply(lambda x : std_convert(x))


STD_TOBZ_M_dic = dict(zip(STD_TOBZ_M_df['고용업종코드(중분류)'], STD_TOBZ_M_df['고용업종명(중분류)']))

STD_TOBZ_M_df

,연번,고용업종코드(대분류),고용업종명(대분류),고용업종코드(중분류),고용업종명(중분류),고용업종코드(소분류),고용업종명(소분류),고용업종코드(세세분류),고용업종명(세세분류)
0,1,A,농업 임업 및 어업,01,농업,11.0,작물 재배업,1110,곡물 및 기타 식량작물 재배업
1,2,A,농업 임업 및 어업,01,농업,11.0,작물 재배업,1121,채소작물 재배업
2,3,A,농업 임업 및 어업,01,농업,11.0,작물 재배업,1122,화훼작물 재배업
3,4,A,농업 임업 및 어업,01,농업,11.0,작물 재배업,1123,종자 및 묘목 생산업
4,5,A,농업 임업 및 어업,01,농업,11.0,작물 재배업,1131,과실작물 재배업
...,...,...,...,...,...,...,...,...,...
1192,1193,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동,97,가구 내 고용활동,970.0,가구 내 고용활동,97000,가구 내 고용활동
1193,1194,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동,98,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,981.0,자가 소비를 위한 가사 생산 활동,98100,자가 소비를 위한 가사 생산 활동
1194,1195,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동,98,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,982.0,자가 소비를 위한 가사 서비스 활동,98200,자가 소비를 위한 가사 서비스 활동
1195,1196,U,국제 및 외국기관,99,국제 및 외국기관,990.0,국제 및 외국기관,99001,주한 외국 공관


In [4]:
code = []
name = []
for i,row in STD_TOBZ_M_df.iterrows():
    code.append(row['고용업종코드(중분류)'])
    name.append(row['고용업종명(중분류)'])
else:
    std_df = pd.DataFrame({'표준산업분류코드':code,'표준산업분류(중)':name})
std_df.drop_duplicates().reset_index(drop=True)

,표준산업분류코드,표준산업분류(중)
0,01,농업
1,02,임업
2,03,어업
3,05,석탄 원유 및 천연가스 광업
4,06,금속 광업
...,...,...
72,95,개인 및 소비용품 수리업
73,96,기타 개인 서비스업
74,97,가구 내 고용활동
75,98,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동


In [106]:
# age전처리하는 함수 정의
def exc_age(age):
    age = age.split(".")[-1].split("대")[0]
    try:
        return "60+" if int(age) >= 60 else age
    except:
        return age

km_data = pd.read_csv("dataset/국민카드.csv", encoding='cp949')
km_data = km_data[~km_data['표준산업세세분류코드'].isna()]
km_data['표준산업세세분류코드'] = km_data['표준산업세세분류코드'].astype(int).astype(str)
km_data['표준산업세세분류코드'] = km_data['표준산업세세분류코드'].apply(lambda x : x[:3])
km_data['STD_TOBZ_M_C'] = km_data['표준산업세세분류코드'].apply(lambda x : x[:2])
km_data['기준년월'] = pd.to_datetime(km_data['기준년월'], format='%Y%m')
km_data['연령대'] = km_data['연령대'].apply(exc_age)
km_data['company'] = '국민'
km_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'code_s_name', 'code_s', 'cost', 'case', 'client', 'code_m', 'company']
km_data = km_data[['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']]
km_data

,date,city_l,city_m,city_s,sex,age,company,code_m,code_s,cost,case
0,2023-04-01,서울특별시,중구,다산동,여성,50,국민,56,561,24370810,683
1,2023-05-01,서울특별시,중구,약수동,남성,60+,국민,59,591,20000,3
2,2023-08-01,서울특별시,중구,신당5동,남성,60+,국민,47,478,129200,3
3,2023-08-01,경기도,동두천시,불현동,여성,30,국민,47,478,79500,7
4,2023-04-01,경기도,동두천시,생연1동,여성,20,국민,91,912,154900,17
...,...,...,...,...,...,...,...,...,...,...,...
292384,2023-03-01,서울특별시,중구,동화동,여성,40,국민,75,759,63600,4
292385,2023-09-01,전라남도,나주시,이창동,남성,40,국민,65,651,2683970,6
292386,2023-10-01,서울특별시,중구,명동,남성,30,국민,47,478,482810,11
292387,2023-03-01,서울특별시,중구,신당동,남성,10,국민,56,561,333700,32


In [118]:
km_data[km_data['city_m']=="중구"]['code_m'].value_counts()

code_m
47    55516
56    20777
49     8527
91     7906
96     7425
86     5767
65     4531
61     4509
52     4007
75     2787
55     2645
84     2046
95     2013
85     1819
10     1660
73     1398
35     1308
90     1231
33     1153
51      964
68      925
59      913
71      666
58      323
60      304
94      189
76      158
45      121
46       65
29       61
41       58
50       36
42       18
Name: count, dtype: int64

In [113]:
km_data['sex'].value_counts()

sex
남성    146957
여성    132610
법인     12619
Name: count, dtype: int64

In [112]:
km_data['age'].value_counts()

age
60+    67043
40     51150
50     50902
30     50469
20     48964
법인     12619
10     11039
Name: count, dtype: int64

In [ ]:
km_count_dic = dict(zip(km_data['STD_TOBZ_M_C'].value_counts().index,km_data['STD_TOBZ_M_C'].value_counts().values))
km_std = pd.DataFrame({'표준산업분류코드':sorted(km_data['STD_TOBZ_M_C'].unique())})
km_std['표준산업분류(중)'] = km_std['표준산업분류코드'].map(STD_TOBZ_M_dic)
km_std['데이터건수'] = km_std['표준산업분류코드'].map(km_count_dic)
km_std.sort_values(by='표준산업분류코드').reset_index(drop=True)

In [110]:
city_l_dic = {'전남':'전라남도', '경기':'경기도', '서울':'서울특별시'}
sex_dic = {'F':'여성', 'M':'남성'}

sh_data = pd.read_csv("dataset/신한카드.csv")
sh_data['MGPO_NM_HOM'] = sh_data['MGPO_NM_HOM'].map(city_l_dic)
sh_data['SEX_C'] = sh_data['SEX_C'].map(sex_dic)
sh_data['SL_CRI_YM'] = pd.to_datetime(sh_data['SL_CRI_YM'], format='%Y%m')
sh_data['STD_TOBZ_M_C'] = sh_data['STD_TOBZ_M_C'].astype(str)
sh_data['company'] = '신한'
sh_data = sh_data[['SL_CRI_YM', 'MGPO_NM_HOM', 'SGG_NM_HOM', 'NASD_NM_HOM', 'SEX_C', 'AGE_G5_C', 'company', 'STD_TOBZ_M_C', 'STD_TOBZ_L_C','AMT', 'CNT']]
sh_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']
sh_data

,date,city_l,city_m,city_s,sex,age,company,code_m,code_s,cost,case
0,2022-12-01,전라남도,나주시,공산면,여성,50,신한,86,861,60000,4
1,2022-12-01,경기도,동두천시,불현동,남성,10,신한,86,861,10000,4
2,2022-12-01,전라남도,나주시,금천면,여성,20,신한,86,861,720000,8
3,2022-12-01,전라남도,나주시,영강동,여성,30,신한,86,861,270000,13
4,2022-12-01,전라남도,나주시,영산동,여성,40,신한,86,861,3460000,12
...,...,...,...,...,...,...,...,...,...,...,...
190553,2023-08-01,전라남도,나주시,금남동,남성,40,신한,59,591,10000,*
190554,2023-02-01,경기도,동두천시,생연1동,남성,40,신한,59,591,30000,*
190555,2023-02-01,전라남도,나주시,왕곡면,여성,40,신한,59,591,10000,*
190556,2023-10-01,경기도,동두천시,상패동,남성,60+,신한,59,591,140000,*


In [ ]:
sh_count_dic = dict(zip(sh_data['STD_TOBZ_M_C'].value_counts().index,sh_data['STD_TOBZ_M_C'].value_counts().values))
sh_std = pd.DataFrame({'표준산업분류코드':sorted(sh_data['STD_TOBZ_M_C'].unique())})
sh_std['표준산업분류(중)'] = sh_std['표준산업분류코드'].map(STD_TOBZ_M_dic)
sh_std['데이터건수'] = sh_std['표준산업분류코드'].map(sh_count_dic)
sh_std.sort_values(by='표준산업분류코드').reset_index(drop=True)

In [111]:
# age전처리하는 함수 정의
def exc_age(age):
    return "60+" if int(age) >= 60 else age
        
ss_data = pd.read_csv("dataset/삼성카드.csv")
ss_data['표준업종분류코드_중분류'] = ss_data['표준업종분류코드_중분류'].astype(str)
ss_data['거주지_광역시도'] = ss_data['거주지_광역시도'].map(city_l_dic)
ss_data['성별'] = ss_data['성별'].map(sex_dic)
ss_data['이용기준년월'] = pd.to_datetime(ss_data['이용기준년월'], format='%Y%m')
ss_data['연령대'] = ss_data['연령대'].apply(exc_age)
ss_data['company'] = '신한'
ss_data = ss_data[['이용기준년월', '거주지_광역시도', '거주지_시군구', '거주지_행정동', '성별', '연령대', 'company', '표준업종분류코드_중분류', '표준업종분류코드_소분류','이용금액', '이용건수']]
ss_data.columns = ['date', 'city_l', 'city_m', 'city_s', 'sex', 'age', 'company', 'code_m', 'code_s', 'cost', 'case']
ss_data

,date,city_l,city_m,city_s,sex,age,company,code_m,code_s,cost,case
0,2023-05-01,경기도,동두천시,소요동,남성,50,신한,86,861,11000,2
1,2023-05-01,전라남도,나주시,금남동,남성,30,신한,86,861,18000,2
2,2023-11-01,경기도,동두천시,중앙동,남성,30,신한,86,861,600000,2
3,2023-07-01,경기도,동두천시,보산동,여성,30,신한,86,861,23000,2
4,2023-10-01,서울특별시,중구,장충동,남성,60+,신한,86,861,14000,2
...,...,...,...,...,...,...,...,...,...,...,...
167170,2023-01-01,서울특별시,중구,소공동,여성,30,신한,90,902,31000,2
167171,2023-03-01,경기도,동두천시,송내동,여성,20,신한,90,902,54000,2
167172,2023-11-01,경기도,동두천시,생연1동,남성,50,신한,90,902,8000,2
167173,2023-03-01,서울특별시,중구,중림동,남성,40,신한,90,902,137000,13


In [10]:
ss_count_dic = dict(zip(ss_data['표준업종분류코드_중분류'].value_counts().index,ss_data['표준업종분류코드_중분류'].value_counts().values))
ss_std = pd.DataFrame({'표준산업분류코드':sorted(ss_data['표준업종분류코드_중분류'].unique())})
ss_std['표준산업분류(중)'] = ss_std['표준산업분류코드'].map(STD_TOBZ_M_dic)
ss_std['데이터건수'] = ss_std['표준산업분류코드'].map(ss_count_dic)
ss_std.sort_values(by='표준산업분류코드').reset_index(drop=True)

,표준산업분류코드,표준산업분류(중),데이터건수
0,10,식료품 제조업,148
1,14,의복 의복 액세서리 및 모피제품 제조업,106
2,18,인쇄 및 기록매체 복제업,1117
3,29,기타 기계 및 장비 제조업,2229
4,30,자동차 및 트레일러 제조업,5
5,33,기타 제품 제조업,327
6,34,산업용 기계 및 장비 수리업,22
7,42,전문직별 공사업,809
8,45,자동차 및 부품 판매업,427
9,46,도매 및 상품 중개업,6075


In [11]:
km_lst = sorted(km_std['표준산업분류코드'].unique())
sh_lst = sorted(sh_std['표준산업분류코드'].unique())
ss_lst = sorted(ss_std['표준산업분류코드'].unique())

In [12]:
std_lst = sorted(list(set(km_std['표준산업분류코드'])|set(sh_std['표준산업분류코드'])|set(ss_std['표준산업분류코드'])))

In [13]:
code = []
km = []
sh = []
ss = []
for i in std_lst:
    code.append(i)
    if i in km_lst:
        km.append('O')
    else:
        km.append('X')
    if i in sh_lst:
        sh.append('O')
    else:
        sh.append('X')
    if i in ss_lst:
        ss.append('O')
    else:
        ss.append('X')

name = []
for i in code:
    name.append(STD_TOBZ_M_dic[i])

existent_df = pd.DataFrame({'표준산업분류코드':code, '표준산업분류(중)':name, '국민':km, '신한':sh, '삼성':ss})
existent_df

,표준산업분류코드,표준산업분류(중),국민,신한,삼성
0,10,식료품 제조업,O,X,O
1,14,의복 의복 액세서리 및 모피제품 제조업,X,X,O
2,18,인쇄 및 기록매체 복제업,X,O,O
3,21,의료용 물질 및 의약품 제조업,X,O,X
4,29,기타 기계 및 장비 제조업,O,O,O
5,30,자동차 및 트레일러 제조업,X,X,O
6,33,기타 제품 제조업,O,O,O
7,34,산업용 기계 및 장비 수리업,X,O,O
8,35,전기 가스 증기 및 공기 조절 공급업,O,X,X
9,41,종합 건설업,O,X,X


In [40]:
existent_df[(existent_df['국민'] == 'O') & (existent_df['신한'] == 'O') & (existent_df['삼성'] == 'O')].to_excel('카드사_표준산업분류_공통항목.xlsx', index=False)

In [36]:
existent_df.to_excel('표준산업분류코드_존재여부2.xlsx', index=False)